In [1]:
%%capture --no-stderr
!pip install langchain-ollama
!pip install langchain-nomic "nomic[local]"
!pip install langchain
!pip install tiktoken scikit-learn
!pip install --quiet -U langchain langchain_community tiktoken langchain-nomic "nomic[local]" langchain-ollama scikit-learn langgraph tavily-python bs4

In [2]:
from langchain_ollama import ChatOllama
model_id = "llama3.2:1b"
llm = ChatOllama(model=model_id, temperature=0)

### LLM


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_nomic.embeddings import NomicEmbeddings
from langchain_ollama import ChatOllama

urls = [
    "https://www.wcpss.net/site/default.aspx?PageType=3&DomainID=15369&ModuleInstanceID=82538&ViewID=6446EE88-D30C-497E-9316-3F8874B3E108&RenderLoc=0&FlexDataID=326169&PageID=39173",
    "https://www.wcpss.net/site/default.aspx?PageType=3&DomainID=15369&ModuleInstanceID=82538&ViewID=6446EE88-D30C-497E-9316-3F8874B3E108&RenderLoc=0&FlexDataID=325688&PageID=39173",
    "https://www.wcpss.net/site/default.aspx?PageType=3&DomainID=15369&ModuleInstanceID=82538&ViewID=6446EE88-D30C-497E-9316-3F8874B3E108&RenderLoc=0&FlexDataID=325422&PageID=39173",
    "https://www.wcpss.net/site/default.aspx?PageType=3&DomainID=15369&ModuleInstanceID=82538&ViewID=6446EE88-D30C-497E-9316-3F8874B3E108&RenderLoc=0&FlexDataID=325129&PageID=39173"
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=200
)
doc_splits = text_splitter.split_documents(docs_list)

vectorstore = SKLearnVectorStore.from_documents(
    documents=doc_splits,
    embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local"),
)

retriever = vectorstore.as_retriever(k=3)



USER_AGENT environment variable not set, consider setting it to identify your requests.
Embedding texts: 100%|██████████| 30/30 [01:54<00:00,  3.81s/inputs]


In [4]:
print(model)

NameError: name 'model' is not defined

In [5]:
### Generate
question = "When does the book club meet"
# Prompt
rag_prompt = """You are an assistant for question-answering tasks about Green Level High School. 

Here is the context to use to answer the question:

{context} 

Think carefully about the above context. 

Now, review the user question:

{question}

Provide an answer to this questions using only the above context. 

Use three sentences maximum and keep the answer concise.

Answer:"""


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Test
docs = retriever.invoke(question)
docs_txt = format_docs(docs)
rag_prompt_formatted = rag_prompt.format(context=docs_txt, question=question)
generation = llm.invoke([HumanMessage(content=rag_prompt_formatted)])
print(generation.content)

Embedding texts: 100%|██████████| 1/1 [00:00<00:00, 10.81inputs/s]


NameError: name 'HumanMessage' is not defined